- 要用 F12 的檢查功能找API
    - offset 是以80為單位去抓產品清單資料
- 產品連結是https://goods.ruten.com.tw/item/show? + ID
- 商品描述藏在以下的連結中，但我們並不知道當中的k參數、o參數要怎麼設定，
    https://goods.ruten.com.tw/item/goods_comments.php?id=22013384838237&k=20e83a8b&o=1591923102
- 商品描述要加refer 才能request到資料

還好查詢過後可以在原先的 descriptionUrl 中找到，這時候我們就可以透過這個資訊抓資料囉!

In [20]:
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [90]:
headers = {'User-Agent':'GoogleBot'}

In [200]:
# 商品分類
cateid = '001100020027'
pages = 50

prodids = []

for page in list(range(1, pages)):
    offset = 1 + 80*page
    url = 'https://rtapi.ruten.com.tw/api/search/v3/index.php/core/prod?cateid={}&sort=rnk%2Fdc&offset={}&limit=80&2653512&_callback=jsonpcb_CoreProd'.format(cateid, offset)
    resp = requests.get(url, headers=headers)
    data = re.sub(r'try\{jsonpcb_CoreProd\(|\);\}catch\(e\)\{if\(window.console\)\{console.log\(e\);\}\}','', resp.text)
    for prod in json.loads(data)['Rows']:
        prodids.append(prod['Id'])
    prodids = list(set(prodids))
    print('There are {} prods in list.'.format(len(prodids)))

There are 80 prods in list.
There are 160 prods in list.
There are 240 prods in list.
There are 320 prods in list.
There are 400 prods in list.
There are 480 prods in list.
There are 560 prods in list.
There are 640 prods in list.
There are 720 prods in list.
There are 800 prods in list.
There are 880 prods in list.
There are 960 prods in list.
There are 1040 prods in list.
There are 1120 prods in list.
There are 1200 prods in list.
There are 1280 prods in list.
There are 1360 prods in list.
There are 1440 prods in list.
There are 1520 prods in list.
There are 1600 prods in list.
There are 1680 prods in list.
There are 1760 prods in list.
There are 1840 prods in list.
There are 1920 prods in list.
There are 2000 prods in list.
There are 2080 prods in list.
There are 2160 prods in list.
There are 2240 prods in list.
There are 2320 prods in list.
There are 2400 prods in list.
There are 2480 prods in list.
There are 2560 prods in list.
There are 2640 prods in list.
There are 2720 prods in

In [201]:
df = []
for i, prodid in enumerate(prodids):
    print('='*60)
    print('Dealing with {}: {}'.format(i, prodid))
    headers = {'User-Agent':'GoogleBot'}
    url = 'https://goods.ruten.com.tw/item/show?' + prodid
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text)
    data1 = json.loads(soup.findAll('script',{'type':'application/ld+json'})[0].text)
    data2 = json.loads(soup.findAll('script',{'type':'application/ld+json'})[1].text)
    data3 = json.loads(soup.findAll('script',{'type':'application/ld+json'})[2].text)
    # 產品名稱
    print('名稱：', data1['name'])
    # 幣別
    print('幣別：', data1['offers']['priceCurrency'])
    # 價格
    print('價格：', data1['offers']['price'])
    # 商品分類
    print('商品分類：', r'/'.join([i['item']['name'] for i in data3['itemListElement']]))
    # 商品描述
    descurl = re.findall(r'goods_comments.php\?id=.*?o=[0-9]{1,}',resp.text)[0]
    descurl = re.sub('&amp;','&',descurl)
    descurl = 'https://goods.ruten.com.tw/item/' + descurl
    headers = {'User-Agent':'GoogleBot',
               'Referer': 'https://goods.ruten.com.tw/item/show?{}'.format(prodid)}
    resp2 = requests.get(descurl, headers=headers)
    soup2 = BeautifulSoup(resp2.text)
    desc = ' '.join(i.text for i in soup2.findAll('p'))
    
    ndf = pd.DataFrame([{'name': data1['name'],
                         'description':data1['description'],
                         'productId':data1['productId'],
                         'brand_name':data1['brand']['name'],
                         'priceCurrency':data1['offers']['priceCurrency'],
                         'price':data1['offers']['price'],
                         'contentLocation':data2['contentLocation'],
                         'item_name':r'/'.join([i['item']['name'] for i in data3['itemListElement']]),
                         'describe':desc}])
    df.append(ndf)

df = pd.concat(df, ignore_index=True)
df

Dealing with 0: 30202516960374
Acer NX.VK9AA.002
TWD
58137
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 1: 30202475013175
HP 8460W DSC WLCD HD+ FLT WCAM RCTO UNIT - 650693-001
TWD
3422
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 2: 30202467452500
Latitude•4GB•Intel i5-4200M 2.5 GHz•W10P•Silver•HD HD•500GB•6Cell BatteryWh•DVD
TWD
14991
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3: 30201122038754
Microsoft Surface Pro 1st Gen 1514 i5-3317U 1.7GHz 128GB SSD 4GB Win 10 Pro
TWD
4869
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 4: 30202408707171
WILLIAMS BLUE BIRD 2 ( BB2 ) CPU With MAYAN SUN SOFTWARE
TWD
10964
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 5: 30201274725137
Avita Clarus 14&quot; Laptop Intel i5-7Y54 1.2GHz 8GB RAM -BOOTS -RR
TWD
5192
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 6: 30202416479097
Dell Latitude E6410 Core i5 / PARTS OR REPAIR/ WINDOWS 10 PROF
TWD
3880
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 7: 30202545770881
Lenovo 20S00030US Ts T14 G1 I5 8g 256g W10p
TWD
5222

Dealing with 41: 30202429509578
Elitebook Revolve•HD HD•W10H•120GB•8GB•Refurbished•ionCell IonWh•WARRANTY
TWD
16257
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 42: 30202402456794
ASUS Chromebook Flip 10.1&quot; Rockchip RK3399 4GB 32GB (C101PA-DS04) *READ*
TWD
7163
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 43: 30201695550848
Dell Latitude E6530 i7 2.9 Ghz 8GB NEW 240GB SSD Win 10 Pro x64 Office2019
TWD
12272
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 44: 30202456730888
LENOVO TOPSELLER DT 11AD0028US TOPSELLER TC M90N-1 I3-8145U
TWD
22432
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 45: 30202417207985
HP EliteBook 745 G6 14 Notebook - 1920 x 1080 - Ryzen 5 3500U - 8 GB RAM - 256
TWD
36135
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 46: 30202451335233
Dell Latitude LAPTOP INTEL i5 8GB 350GB SSD HDD HDMI Windows 10 WiFi GAMING PC
TWD
6566
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 47: 30202544228192
Asus laptop 14.5 In Grey with fingerprint scanner on the side, i7 processor.
TWD
23562


Dealing with 81: 30202529468944
Lenovo Thinkpad Yoga 12.5&quot; Laptop Intel Core i5-4300U 1.90GHz 4GB RAM 128GB SSD
TWD
5026
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 82: 30202475207844
HP Envy x360 15m-dr1011dx Convertible Laptop Touch 15t 15.6&quot; i5-10210U 8GB 256GB
TWD
20397
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 83: 30202500507037
HP Probook 450 G2 J8U86UT#ABA i3 4005U 1.7GHz 4GB 500 GB
TWD
13509
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 84: 30202448070505
Samsung 11.6&quot; Chromebook XE501C13-K01US - Intel Celeron 2GB RAM 16GB eMMC
TWD
8168
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 85: 30202423370062
X1 Carbon•8GB•Gray•HD Full•W10•Refurbished•WARRANTY
TWD
25447
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 86: 30202504269272
Alienware AW17R4-7345SLV-PUS 17&quot; Laptop (7th Generation Intel Core i7, 16GB RAM,
TWD
54224
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 87: 30202513198677
HP Stream 14 14 Laptop AMD A4 4GB RAM 32GB eMMC Brilliant Black+ Microsoft Offi
TWD
942

Dealing with 120: 30202509046449
Lenovo ThinkPad T480 20L5004HUS 14 Notebook - 1366 x 768 - + Office 365 Bundle
TWD
27121
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 121: 30184916340829
Microsoft Surface Go 10&quot; Touchscreen Intel Pentium Gold 4GB RAM 64GB SSD Win 10
TWD
10022
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 122: 30202491238273
E5440•14&quot;•240GB•8GB•Intel i5-4310U 2.0 GHz•W10P•DVD•Refurbished•WARRANTY
TWD
14011
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 123: 30202417474510
NEW Lenovo 2-in-1 C340 11.6&quot; HD Touch Intel N4000 32GB eMMC 4GB RAM Chrome OS
TWD
9927
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 124: 30202439366200
HP INC. - SB THINCLIENTS 7RR02UT#ABA SMART BUY MT45 R3-3300U 14IN 8GB 128GB
TWD
35666
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 125: 30202519673952
ThinkPad•Black•8GB•500GB•HD HD•lithiumCell LithiumWh•DVD•Refurbished•WARRANTY
TWD
15849
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 126: 30202493747135
Latitude 3400•FHD 14&quot;•256GB•Intel i5-826

Dealing with 159: 30202243996842
Gigabyte Max - Q Aorus X5 MD-KL4K3M 15.6&quot; UHD IPS Intel Core i7-7820HK GTX Max Q
TWD
51836
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 160: 30202506124814
Toshiba Chromebook 2 Intel i3-5015u IPS FHD HDMI USB 3.0 CB35-C3350 Thin AirBook
TWD
9268
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 161: 30202464405246
Microsoft Surface Pro 1516 32GB NVIDIA TEGRA 3 QUAD CORE@1.30GHz 2GB RAM
TWD
3770
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 162: 30202511113289
Panasonic Toughbook CF-53 Core i5 2 GHz Laptop 500GB 8GB Windows 10 Pro 64
TWD
10210
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 163: 30202395363983
LENOVO - TOPSELLER THINK 20LQ000MUS TPSLLR THNKPD 11E INTEL N5000
TWD
24127
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 164: 30202432417255
HP ProBook 4430S 14&quot; LaptopIntel Core i5-2450M 4GB RAM 250GB HDD Windows 10
TWD
3079
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 165: 30202518077319
E7450•14&quot;•Intel i5-5300U 2.30 GHz•128GB•W10P•8GB•Refurbishe

...

Dealing with 3792: 30202456899716
Portege Z30-C•Wide•Gray•W10•8GB•Refurbished•WARRANTY
TWD
21403
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3793: 22017480464389
☺小咪電舖【有貨宅配免運】╭☆微軟Surface Laptop 3砂岩金(Ci5-1035G7/8G/256G/W10)
TWD
38900
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3794: 30202266209990
BRAND NEW LG gram 13Z990-A.AAS5U1 13.3&quot; laptop FHD Touchscreen i5 8GB 256GB SSD
TWD
26005
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3795: 30202517879509
HP EliteBook 830 G6 13.3 Notebook - 1920 x 1080 - Core i5 i + Office 365 Bundle
TWD
46243
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3796: 30202430356435
Latitude 14” 5000 5400•256GB•M2•NVME•FHD 14&quot;•W10P•i7-8665U•16GB•4Cell 68Wh
TWD
40272
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3797: 30202522245052
Lenovo G550 Intel Core 2 Duo T6600 2.2 GHz 1 GB RAM NO HDD LAPTOP *Parts
TWD
720
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3798: 30202524965604
Asus ROG GU501GM-BI7N8 15.6&quot; FHD i7-8750H 2.2GHz 16GB 1TB+128GB 1060 W10 - Dents


Dealing with 3831: 30202495730791
Panasonic ToughBook CF-53 Core i5-3320M 2.60GHz 2GB WiFi DVD/Rom / No Hard Drive
TWD
4712
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3832: 30202508688596
Dell Vostro 5390 13.3&quot; FHD i5-8265U 1.6GHz 8GB 256GB SSD W10H Laptop U
TWD
17278
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3833: 30202479256853
Fitbit FB4 Inspire HR Heart Rate Fitness Tracker Small - BLACK -
TWD
2482
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3834: 30202514176269
DELL LATITUDE E6430 HD LAPTOP WIN 10 Pro HDMI Webcam INTEL i5 2.7GHz 4GB 500GB
TWD
9268
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3835: 30202417399842
Dell Latitude D620 for parts - bad LCD - 1.66GHz CPU / 2.5GB RAM / 160GB HDD
TWD
628
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3836: 30202479906877
Dell Latitude 5480 i7-7600U 256GB SSD 14&quot; 16GB Windows 10 Pro
TWD
24757
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3837: 30202534257520
HP Zbook 15 15.6&quot; Intel i7 4800MQ@2.7GHz 8GB RAM 128GB SSD Win 10 No Ba

Chrome•4GB•Black•Chrome•16GB•HD HD•Refurbished•WARRANTY
TWD
8089
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3871: 30202393825984
Latitude 14•14 inches•i5-5300U•1366 x 768 HD•W10P•16GB•Black•Widescreen
TWD
21485
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3872: 30201662527723
Dell Latitude E6500 Core 2 Duo @ 2.40GHz 250GB HDD 4GB RAM Linux Mint 19
TWD
4398
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3873: 30202528508735
New 10.1&quot; Notebook Laptop Quad Core Android 6 1.5 GHZ with front camera Offer!!
TWD
3644
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3874: 21945796189045
5Cgo【聯強】acer TravelMate P2/i5-8250U 14吋筆電 TMP2410-G2-MG-54CR
TWD
26200
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3875: 30202464894710
Acer Chromebook 15
TWD
16201
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3876: 30202544289751
HP Chromebook 11.6&quot; HD 4GB Google Chrome Laptop Computer - Black
TWD
8796
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3877: 30202457398205
Dell Inspiron 15 5000 15.6in AMD A8-7410 Qu

Dealing with 3910: 30202404780538
Dell Latitude 12&quot; E6230 i7-3520M 2.9GHz 4GB 256GB SSD Backlit WiFi Win 7 Laptop
TWD
6597
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3911: 30202485277561
Microsoft Surface Pro 4 8GB Windows 10 Pro 256GB SSD Intel Core i5
TWD
10367
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3912: 30202390431951
Microsoft Surface Go 128GB SSD, 8GB RAM, Silver MCZ-00001 Over $600 Package
TWD
13652
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3913: 30202538056228
MT PC LENOVO ThinkCentre M73 Intel i3-4130. 4GB/500GB, No OS &quot;A&quot;
TWD
4083
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3914: 30202428333257
HP 15-dw0043dx 15.6&quot; Touch HD- Core i5 / 8GB memory 128GB SSD Pre Owned laptop
TWD
14137
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3915: 30202525425808
Lenovo IdeaPad 530S AMD Ryzen 5 2500U 2.0GHz 14&quot; 8GB 256GB SSD Windows 10
TWD
16251
露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電 
Dealing with 3916: 30202456803036
Dell Latitude D610 1.6GHz Intel Pentium M 1.GB RAM 4

,name,description,productId,brand_name,priceCurrency,price,contentLocation,item_name,describe
0,Acer NX.VK9AA.002,"直購價：58,137元。物品狀態：全新。支付方式包含PChomePay支付連。(302025...",30202516960374,interloper-com@ebay,TWD,58137,"US,Dallas",露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電,
1,HP 8460W DSC WLCD HD+ FLT WCAM RCTO UNIT - 650...,"直購價：3,422元。物品狀態：請參考商品說明。支付方式包含PChomePay支付連。(30...",30202475013175,wicky_dog@ebay,TWD,3422,"US,Syosset",露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電,
2,Latitude•4GB•Intel i5-4200M 2.5 GHz•W10P•Silve...,"直購價：14,991元。物品狀態：請參考商品說明。支付方式包含PChomePay支付連。(3...",30202467452500,swingcomputers@ebay,TWD,14991,"US,Redwood City",露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電,
3,Microsoft Surface Pro 1st Gen 1514 i5-3317U 1....,"直購價：4,869元。物品狀態：請參考商品說明。支付方式包含PChomePay支付連。(30...",30201122038754,powercellularinc@ebay,TWD,4869,"US,Northbrook",露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電,
4,WILLIAMS BLUE BIRD 2 ( BB2 ) CPU With MAYAN SU...,"直購價：10,964元。物品狀態：請參考商品說明。支付方式包含PChomePay支付連。(3...",30202408707171,babydoll_gaming@ebay,TWD,10964,"US,Houston",露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電,***Check out my Ebay Store /babydollgamingmo...
...,...,...,...,...,...,...,...,...,...
3915,Lenovo IdeaPad 530S AMD Ryzen 5 2500U 2.0GHz 1...,"直購價：16,251元。物品狀態：請參考商品說明。支付方式包含PChomePay支付連。(3...",30202525425808,highlido@ebay,TWD,16251,"US,Linden",露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電,Lenovo IdeaPad 530S AMD Ryzen 5 2500U 2.0GH...
3916,Dell Latitude D610 1.6GHz Intel Pentium M 1.GB...,"直購價：2,670元。物品狀態：請參考商品說明。支付方式包含PChomePay支付連。(30...",30202456803036,nookdesignbysierra@ebay,TWD,2670,"US,Caliente",露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電,
3917,Custom Build your Panasonic Toughbook CF-30 Ru...,"直購價：16,922元。物品狀態：請參考商品說明。支付方式包含PChomePay支付連。(3...",30202452257057,rugged.laptops@ebay,TWD,16922,"CA,Scarborough",露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電,
3918,Toshiba Tecra 730XCDT Laptop,"直購價：2,513元。物品狀態：請參考商品說明。支付方式包含PChomePay支付連。(30...",30202512726224,david6007@ebay,TWD,2513,"US,York",露天拍賣/ 電腦、電子、周邊 / 筆記型電腦 / 商用筆電,


In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3920 entries, 0 to 3919
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             3920 non-null   object
 1   description      3920 non-null   object
 2   productId        3920 non-null   object
 3   brand_name       3920 non-null   object
 4   priceCurrency    3920 non-null   object
 5   price            3920 non-null   object
 6   contentLocation  3920 non-null   object
 7   item_name        3920 non-null   object
 8   describe         3920 non-null   object
dtypes: object(9)
memory usage: 275.8+ KB


In [203]:
df.to_excel('./RuTen.xlsx')